# Data Retrieval - Jupyter Notebook Generator

In [ ]:
# Install the required libraries for newly created environment - only once

#!pip install pandas
#!pip install sqlalchemy
#!pip install pymysql
#!pip install sqlparse

In [ ]:
# Import required libraries
import base64
import os
import sqlalchemy
import sqlparse

import pandas as pd
import nbformat as nbf

from sqlalchemy import create_engine
from nbformat.v4 import new_code_cell, new_markdown_cell

In [ ]:
# Configure username, password and database details
db_user = "<<user_name>>"
db_password = "<<user_password>>"
db_host = "<<host_name>>"
db_port = "<<port>>"
db_name = "<<database_name>>"

In [ ]:
# Connection details
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

In [ ]:
# Establish the connection - uncomment to validate connection
# engine = create_engine(connection_string)

In [ ]:
# Function to prepare the schema information in table and its index details
def prepare_schema_info(connection):
    tables_df = pd.read_sql("SHOW TABLES;", connection)
    tables = tables_df.iloc[:, 0]

    schema_info = {}
    for table in tables:
        indexes_df = pd.read_sql(f"SHOW INDEXES FROM {table};", connection)
        schema_info[table] = indexes_df

    return schema_info

In [ ]:
# Get the schema information - Uncomment to validate
# schema_info = prepare_schema_info(engine)
# schema_info

In [ ]:
# Function to prepare vairous SELECT queries based on indexes (including composite indexes) with parameterized queries
def prepare_select_queries(schema_info):
    select_queries_map = {}  # Use a dictionary to store queries for each table
    unique_indexes = set()
    
    for table, indexes_df in schema_info.items():
        select_queries_set = set()  # Use a set to store unique queries for each table

        for index_name, group in indexes_df.groupby("Key_name"):
            if index_name in unique_indexes:  # Skip if we have already processed this index
                continue

            unique_indexes.add(index_name)

            non_unique = group["Non_unique"].iloc[0]  # Since it's the same for the whole index

            # Check if it's a composite index with more than one column
            composite_columns = group["Column_name"].tolist()

            if index_name == "PRIMARY" or non_unique == 0:  # Primary key or unique index
                # Create the WHERE clause for composite indexes using the AND operator
                where_clause = " AND ".join([f"{col} = :{col}" for col in composite_columns])
                query = f"SELECT * FROM {table} WHERE {where_clause};"
            else:  # Non-unique index
                query = f"SELECT * FROM {table} WHERE {composite_columns[0]} = :{composite_columns[0]};"

            select_queries_set.add(query)  # Use add() to add unique queries to the set
        
        #if select_queries_set:  # Only add entry to the dictionary if the set is not empty
        select_queries_map[table] = select_queries_set

    return select_queries_map

In [ ]:
# Prepare Table specific queries - Uncomment to validate map
# select_queries_map = prepare_select_queries(schema_info)
# select_queries_map

In [ ]:
header_markdown = "# Data Retrieval Notebook for Table: {table}"

In [ ]:
import_statment_code = f"""# Import required library packages
import base64
import pandas as pd
from sqlalchemy import create_engine, text
"""

In [ ]:
connection_string_code = """# Connection details
connection_string = {connection_string}
"""

In [ ]:
create_engine_code = """# Establish the connection
engine = create_engine(base64.b64decode(connection_string).decode("ascii"))
"""

In [ ]:
format_sql_query_func_code = """# Format SQL Query
def format_sql_query(query):
    formatted_query = sqlparse.format(query, reindent=True, keyword_case="upper")
    return formatted_query.strip()
"""

In [ ]:
exec_sql_query_func_code="""# Execute Select Query
def exec_sql_query(sql=None, con=engine, params=None):
    df = pd.read_sql_query(sql=text(query), con=engine, params=params)
    df_filtered = df.dropna(axis=1, how='all')
    return (df, df_filtered)
"""

In [ ]:
parameter_variable_def_code="# Parameter variables for {table}\n{param_vars_code}"

In [ ]:
select_all_query_code = "SELECT * FROM {table} LIMIT 10;"

In [ ]:
engine_dispose_code = """# Disconnect
engine.dispose()
"""

In [ ]:
exec_sql_code = """# Execute sql query and display
df, df_filtered = exec_sql_query(sql=text(query), con=engine, params=parameters)
"""

In [ ]:
pandas_df_code = """# Display information
df_filtered.head()
"""

In [ ]:
# Function to format given query

def format_sql_query(query):
    formatted_query = sqlparse.format(query, reindent=True, keyword_case="upper")
    return formatted_query.strip()

In [ ]:
def append_markdown_cell(nb, markdown_content):
    nb.cells.append(nbf.v4.new_markdown_cell(source=markdown_content))

In [ ]:
def append_code_cell(nb, code_content):
    nb.cells.append(nbf.v4.new_code_cell(source=code_content))

In [ ]:
def extract_parameters_from_query(query):
    # Split the query into words
    words = query.split()

    # Extract words starting with ':'
    parameters = [word[1:] for word in words if word.startswith(':')]

    # Remove ";" from the parameter values and convert to lowercase
    parameters = [param.rstrip(';') for param in parameters]

    return parameters

In [ ]:
def get_unique_parameters(queries):
    unique_parameters = set()
    for query in queries:
        parameters = extract_parameters_from_query(query)
        unique_parameters.update(parameters)
    return unique_parameters

In [ ]:
def append_query_cells(nb, table, query):
    parameters = extract_parameters_from_query(query)
    #query = query.lower()
    formatted_query = format_sql_query(query)
    
    # Query as a Markdown Cell
    query_markdown = f"```sql\n{formatted_query}\n```"
    append_markdown_cell(nb, query_markdown)

    # Assign SQL query to variable
    query_cell_code = f"query = \\\n\"\"\"\n{formatted_query}\n\"\"\"\n"
    append_code_cell(nb, f"# Query for {table}\n{query_cell_code}")

    # Parameterzied SQL query
    param_str = ", ".join(f'"{param}": {param}' for param in parameters)
    param_cell_code = f"# Parameters for {table}\nparameters = {{{param_str}}}"
    append_code_cell(nb, param_cell_code)

    # Function to execute SQL Query
    append_code_cell(nb, exec_sql_code)
    
    # Display Dataframe by head function
    append_code_cell(nb, pandas_df_code)

In [ ]:
def append_select_all_query_cells(nb, table, unique_parameters):
    if unique_parameters:
        param_vars_code = "\n".join(f"{param} = ''" for param in unique_parameters)
        append_code_cell(nb, f"# Parameter variables for {table}\n{param_vars_code}")

    append_query_cells(nb, table, select_all_query_code.format(table=table))

In [ ]:
def generate_data_retrieval_notebook(table, queries):
    nb = nbf.v4.new_notebook()
    nb.metadata.kernelspec = {
        "name": "python3",
        "display_name": "Python 3",
        "language": "python"
    }
    
    # Header markdown
    append_markdown_cell(nb, header_markdown.format(table=table))
    
    # Import statements
    append_code_cell(nb, import_statment_code)
    
    # Set Connection string
    connection_string = base64.b64encode(f"mysql+pymysql://{username}:{password}@{hostname}:{port}/{database_name}".encode("ascii"))
    append_code_cell(nb, connection_string_code.format(connection_string=connection_string))
    
    # Create Connection
    append_code_cell(nb, create_engine_code)
    
    # Format SQL Query function
    append_code_cell(nb, format_sql_query_func_code)
    
    # Execute SQL Query Function
    append_code_cell(nb, exec_sql_query_func_code)
    
    # Select All Records with Limit
    unique_parameters = get_unique_parameters(queries)
    append_select_all_query_cells(nb, table, unique_parameters)
    
    # For each Query Append Cells
    for query in queries:
        append_query_cells(nb, table, query)
    
    # Disconnect SQL Connection
    append_code_cell(nb, engine_dispose_code)
    
    if not os.path.exists(f'generated/{database_name}'):
        os.makedirs(f'generated/{database_name}')

    file_name = f"generated/{database_name}/{table}.ipynb"
    nbf.write(nb, file_name)

In [ ]:
def generate_data_retrieval_notebooks(select_queries_map):
    for table, queries in select_queries_map.items():
        generate_data_retrieval_notebook(table, queries)

In [ ]:
if __name__ == "__main__":
    print("Generating Data Retrieval Notebooks - Started ...")
    engine = create_engine(connection_string)
    schema_info = prepare_schema_info(engine)
    select_queries_map = prepare_select_queries(schema_info)
    generate_data_retrieval_notebooks(select_queries_map)
    engine.dispose()
    print("Generating Data Retrieval Notebooks - Finished")